# Animal Identification - Keras Model Evaluation

In [1]:
# importing python packages for use

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split

# keras must be installed separately from normal python packages

import keras 
from keras.preprocessing import image as keraimage
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential
from keras.utils import to_categorical

# importing machine learning models to test

import sklearn.ensemble as ske
RFC = ske.RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.svm import SVC

# importing metrics for model evaluation

from sklearn import metrics

Using TensorFlow backend.


In [2]:
# importing the file with the group classification

train = pd.read_excel('Training_ID.xlsx')
train.head()

,Image,Class,ID
0,0,Mammal,1
1,1,Mammal,1
2,2,Mammal,1
3,3,Mammal,1
4,4,Mammal,1


In [3]:
labels = train['ID']
labels.head()

0    1
1    1
2    1
3    1
4    1
Name: ID, dtype: int64

# Keras Method of Image Classification:
# Convolutional Neural Networks (CNN)

Keras is a package in Python that is used quite often for image classification.
I came accross it while doing research on classification methods and wanted to compare it to my other results.

In [4]:
# finding the average image size

heights = []
widths = []

for j in range(train.shape[0]):
    
    image_path = 'Training_Data/'+train['Image'][j].astype('str')+'.jpg'
    img = Image.open(image_path)
    hxw = np.array(img)
    heights.append(hxw.shape[0])
    widths.append(hxw.shape[1])

mean_h = sum(heights)/len(heights)
mean_w = sum(widths)/len(widths)
print(mean_h)
print(mean_w)

602.235
929.492


In [5]:
# changing the size of each image
# required to work in the keras classification model

img_h = 600
img_w = 930

for k in range(train.shape[0]):
    
    image_path = 'Training_Data/'+train['Image'][k].astype('str')+'.jpg'
    img = Image.open(image_path)
    new_img = img.convert('L')
    new_img = new_img.resize((img_w,img_h), Image.ANTIALIAS)
    
    new_image_path = 'Training_Data/Resized_Images/'+train['Image'][k].astype('str')+'_Resized.jpg'
    new_img.save(new_image_path)

In [7]:
# Pulling the data from the image

raw_image_data = []

for l in range(train.shape[0]):

    image_path = 'Training_Data/Resized_Images/'+train['Image'][l].astype('str')+'_Resized.jpg'
    pic = Image.open(image_path)
    
    pic = keraimage.img_to_array(pic)
    pic = pic/255
    raw_image_data.append(pic)
    
X_keras = np.array(raw_image_data)
Y_keras = to_categorical(labels)

In [8]:
print(X_keras.shape)
print(Y_keras.shape)

(1000, 600, 930, 1)
(1000, 6)


In [9]:
# creating test and training sets for the model

X_train_keras,X_test_keras,Y_train_keras,Y_test_keras = train_test_split(X_keras,Y_keras,test_size = 0.125,random_state = 0)
X_train_keras.shape

(875, 600, 930, 1)

In [10]:
# creating the keras model for image recognition

keras_model = Sequential()
keras_model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=(600,930,1)))
keras_model.add(Conv2D(64,(3,3),activation='relu'))
keras_model.add(MaxPooling2D(pool_size=(2, 2)))
keras_model.add(Dropout(0.25))
keras_model.add(Flatten())
keras_model.add(Dense(128, activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(6, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
keras_model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
keras_model.fit(X_train_keras,Y_train_keras,epochs=1,validation_data=(X_test_keras,Y_test_keras))

Instructions for updating:
Use tf.cast instead.
Train on 875 samples, validate on 125 samples
Epoch 1/1
 32/875 [>.............................] - ETA: 2:28:23 - loss: 1.7964 - accuracy: 0.1250

In [ ]:
Y_keras_predict = keras_model.predict_classes(X_test_keras)